# RNN-LSTM_XAI(ELI5)

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import r2_score
import eli5
from eli5.sklearn import PermutationImportance

# Load the sunspots dataset
sunspots = np.loadtxt("Sunspots.csv", delimiter=',', skiprows=1, usecols=[2])

# Define window size and batch size
window_size = 10
batch_size = 32

# Function to create windowed dataset
def windowed_dataset(series, window_size, batch_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

# Create train and test sets
split_time = 3000
train_series = sunspots[:split_time]
test_series = sunspots[split_time:]
train_set = windowed_dataset(train_series, window_size, batch_size)
test_set = windowed_dataset(test_series, window_size, batch_size)

# Define the RNN-LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=[None, 1]),
    LSTM(64),
    Dense(1)
])

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(train_set, epochs=10, verbose=0)

# Function to create a wrapper class for the model
# Function to create a wrapper class for the model
class ModelWrapper:
    def __init__(self, model):
        self.model = model
        
    def predict(self, X):
        return self.model.predict(X)
    
    def score(self, X, y):
        y_pred = self.model.predict(X)
        return r2_score(y, y_pred)
    
    def fit(self, X, y):
        pass

# Wrap the model
wrapped_model = ModelWrapper(model)

# Extract X_test and y_test
X_test = np.concatenate([X.numpy() for X, y in test_set], axis=0)
y_test = np.concatenate([y.numpy() for X, y in test_set], axis=0)

# Apply permutation importance using Eli5
perm = PermutationImportance(wrapped_model, random_state=42)
perm.fit(X_test, y_test)

# Display permutation importance results
eli5.show_weights(perm)

import matplotlib.pyplot as plt
plt.savefig("eli5 on rnn-lstm")

D:\Anaconda3-2024.02-1-Windows\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
D:\Anaconda3-2024.02-1-Windows\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━

<Figure size 640x480 with 0 Axes>

# GRU_XAI(ELI5)

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import r2_score
import eli5
from eli5.sklearn import PermutationImportance

# Load the sunspots dataset
sunspots = np.loadtxt("Sunspots.csv", delimiter=',', skiprows=1, usecols=[2])

# Define window size and batch size
window_size = 10
batch_size = 32

# Function to create windowed dataset
def windowed_dataset(series, window_size, batch_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

# Create train and test sets
split_time = 3000
train_series = sunspots[:split_time]
test_series = sunspots[split_time:]
train_set = windowed_dataset(train_series, window_size, batch_size)
test_set = windowed_dataset(test_series, window_size, batch_size)

# Define the GRU model
model = Sequential([
    GRU(64, return_sequences=True, input_shape=[None, 1]),
    GRU(64),
    Dense(1)
])

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
model.fit(train_set, epochs=10, verbose=0)

# Function to create a wrapper class for the model
class ModelWrapper:
    def __init__(self, model):
        self.model = model
        
    def predict(self, X):
        return self.model.predict(X)
    
    def score(self, X, y):
        y_pred = self.model.predict(X)
        return r2_score(y, y_pred)
    
    def fit(self, X, y):
        pass

# Wrap the model
wrapped_model = ModelWrapper(model)

# Extract X_test and y_test
X_test = np.concatenate([X.numpy() for X, y in test_set], axis=0)
y_test = np.concatenate([y.numpy() for X, y in test_set], axis=0)

# Apply permutation importance using Eli5
perm = PermutationImportance(wrapped_model, random_state=42)
perm.fit(X_test, y_test)

# Display permutation importance results
eli5.show_weights(perm)

import matplotlib.pyplot as plt
plt.savefig("eli5 on gru")

D:\Anaconda3-2024.02-1-Windows\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
D:\Anaconda3-2024.02-1-Windows\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
8/8 ━━━━━━━━━━

<Figure size 640x480 with 0 Axes>